#Defining the functions

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math

# Define the functions
def f1(x):
    return x**31 - 1000

def f2(x):
    return (np.sin(x))**5 * (np.cos(x))**3 - np.exp(x)

def f3(x):
    return 1/(1 + np.exp(-x**2 - np.sin(x) - x)) - 0.5

# Define the finite difference method to approximate the derivative
def finite_diff(f, x, h=1e-5):
    return (f(x + h) - f(x - h)) / (2 * h)

In [3]:
def calculate_order_of_convergence(errors):
    n = len(errors)
    if n < 2:
        return None  # Not enough data points to determine order
    
    # Calculate differences for log-log plot
    log_errors = np.log(errors)
    log_ratios = np.log(errors[1:] / errors[:-1])

    # Fit a linear regression to find the slope
    slope, intercept = np.polyfit(log_errors[:-1], log_ratios, 1)

    return slope  # This slope is the order of convergence p

#Bisection method

In [4]:
def bisection(f, a, b, tol=1e-3, max_iter=1000):
    if f(a) * f(b) > 0:
        raise ValueError("The function must have different signs at the endpoints a and b.")
    
    iterations = 0
    while (b - a) / 2 > tol and iterations < max_iter:
        c = (a + b) / 2
        if f(c) == 0:
            break
        elif f(a) * f(c) < 0:
            b = c
        else:
            a = c
       
        iterations += 1
    
 
    
    return c, iterations


#Newton-Raphson Method

In [5]:
def newton_raphson(f, x0, tol=1e-3, max_iter=1000):
    x = x0
    iterations = 0
    while abs(f(x)) > tol and iterations < max_iter:
        x_new = x - f(x) / finite_diff(f, x)
        x = x_new
        iterations += 1
        
    
    root = x
  
    
    return root, iterations


#Secant Method

In [6]:
def secant(f, x0, x1, tol=1e-3, max_iter=1000):
    iterations = 0
    while abs(f(x1)) > tol and iterations < max_iter:
        x2 = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        x0, x1 = x1, x2
        iterations += 1
    
    root = x2
    
    return root, iterations


#False Position Method (Regula Falsi)

In [7]:
def false_position(f, a, b, tol=1e-3, max_iter=2000):
    if f(a) * f(b) > 0:
        raise ValueError("The function must have different signs at the endpoints a and b.")
    
    iterations = 0
    while abs(f(b) - f(a)) > tol and iterations < max_iter:
        c = b - f(b) * (b - a) / (f(b) - f(a))
        
        if abs(f(c)) < tol:
            break
        elif f(a) * f(c) < 0:
            b = c
        else:
            a = c
        iterations += 1
    
    root = c
    
    return root, iterations

#function executing all algos

In [8]:
def test_algorithms(f, a, b, x0, x1):
    results = {}
    
    try:
        root_bis, iter_bis = bisection(f, a, b)
        results['Bisection'] = (root_bis, iter_bis)
    except ValueError as e:
        results['Bisection'] = str(e)
    
    try:
        root_nr, iter_nr = newton_raphson(f, x0)
        results['Newton-Raphson'] = (root_nr, iter_nr)
    except Exception as e:
        results['Newton-Raphson'] = str(e)
    
    try:
        root_sec, iter_sec = secant(f, x0, x1)
        results['Secant'] = (root_sec, iter_sec)
    except Exception as e:
        results['Secant'] = str(e)
    
    try:
        root_fp, iter_fp = false_position(f, a, b)
        results['False Position'] = (root_fp, iter_fp)
    except ValueError as e:
        results['False Position'] = str(e)
    
    return results


#Initialise Points

In [9]:
# Define initial guesses and intervals for the test functions
interval_f1 = [1.2, 1.4]
interval_f2 = [-5.4, -2]
interval_f3 = [-2, -0.7]

x0_f1 = 1.5
x1_f1 = 2.0

x0_f2 = -4.5
x1_f2 = -2.0

x0_f3 = -1.2
x1_f3 = -1.0

#Print the root and iterations required

In [10]:
results_f1 = test_algorithms(f1, *interval_f1, x0_f1, x1_f1)
results_f2 = test_algorithms(f2, *interval_f2, x0_f2, x1_f2)
results_f3 = test_algorithms(f3, *interval_f3, x0_f3, x1_f3)

results_f1, results_f2, results_f3

({'Bisection': (1.2484375, 7),
  'Newton-Raphson': (1.2496091495638755, 9),
  'Secant': (1.2496091417004975, 15),
  'False Position': (1.2496091039801553, 119)},
 {'Bisection': (-4.91357421875, 11),
  'Newton-Raphson': (-4.915476118490299, 6),
  'Secant': (-4.908497005018796, 6),
  'False Position': (-4.907858612425408, 4)},
 {'Bisection': (-1.61787109375, 10),
  'Newton-Raphson': (-1.6176351295452107, 3),
  'Secant': (-1.6174428756717847, 5),
  'False Position': (-1.616278984344038, 3)})

#Finding the order of all the four algorithms by passing through all the three functions

In [16]:
def df(x):
    return 31 * x**30

# Root-finding methods
def newton_raphson(f, df, x0, tol=1e-10, max_iter=100):
    x = x0
    errors = []
    for _ in range(max_iter):
        x_new = x - f(x) / df(x)
        errors.append(abs(x_new - x))
        if abs(x_new - x) < tol:
            break
        x = x_new
    return errors

def newton_raphson_numerical(f, x0, tol=1e-10, max_iter=100):
    x = x0
    errors = []
    for _ in range(max_iter):
        x_new = x - f(x) / finite_diff(f, x)
        errors.append(abs(x_new - x))
        if abs(x_new - x) < tol:
            break
        x = x_new
    return errors

def secant(f, x0, x1, tol=1e-10, max_iter=100):
    errors = []
    for _ in range(max_iter):
        if f(x1) == f(x0):  # prevent division by zero
            break
        x_new = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
        errors.append(abs(x_new - x1))
        if abs(x_new - x1) < tol:
            break
        x0, x1 = x1, x_new
    return errors

def bisection(f, a, b, tol=1e-10, max_iter=100):
    errors = []
    for _ in range(max_iter):
        c = (a + b) / 2
        errors.append(abs(b - a))
        if abs(f(c)) < tol or abs(b - a) < tol:
            break
        if f(a) * f(c) < 0:
            b = c
        else:
            a = c
    return errors

def false_position(f, a, b, tol=1e-10, max_iter=100):
    errors = []
    for _ in range(max_iter):
        c = (a * f(b) - b * f(a)) / (f(b) - f(a))
        errors.append(abs(c - b))
        if abs(f(c)) < tol:
            break
        if f(a) * f(c) < 0:
            b = c
        else:
            a = c
    return errors

# Calculate order of convergence
def order_of_convergence(errors):
    rates = 0;
    for i in range(1, len(errors) - 1):
        if errors[i] == 0 or errors[i-1] == 0:  # prevent division by zero
            continue
        rate = np.log(errors[i + 1] / errors[i]) / np.log(errors[i] / errors[i - 1])
        rates+=rate
    return rates/len(errors)

# Define the starting points and intervals
x0_f = 1.5
x0_g = 0.1
x0_h = 0.1
x1_g = 0.2
x1_h = 0.2

a_f, b_f = 1.0, 2.0
a_g, b_g = -5.0, -4.0
a_h, b_h = -2.0, -1.0

# Run the methods for f(x)
nr_errors_f = newton_raphson(f1, df, x0_f)
secant_errors_f = secant(f1, 1.5, 2.0)
bisection_errors_f = bisection(f1, a_f, b_f)
false_pos_errors_f = false_position(f1, a_f, b_f)

# Run the methods for g(x) using numerical derivatives for Newton-Raphson
nr_errors_g = newton_raphson_numerical(f2, x0_g)
secant_errors_g = secant(f2, x0_g, x1_g)
bisection_errors_g = bisection(f2, a_g, b_g)
false_pos_errors_g = false_position(f2, a_g, b_g)

# Run the methods for h(x) using numerical derivatives for Newton-Raphson
nr_errors_h = newton_raphson_numerical(f3, x0_h)
secant_errors_h = secant(f3, x0_h, x1_h)
bisection_errors_h = bisection(f3, a_h, b_h)
false_pos_errors_h = false_position(f3, a_h, b_h)

# Calculate orders of convergence
nr_order_f = order_of_convergence(nr_errors_f)
secant_order_f = order_of_convergence(secant_errors_f)
bisection_order_f = order_of_convergence(bisection_errors_f)
false_pos_order_f = order_of_convergence(false_pos_errors_f)

nr_order_g = order_of_convergence(nr_errors_g)
secant_order_g = order_of_convergence(secant_errors_g)
bisection_order_g = order_of_convergence(bisection_errors_g)
false_pos_order_g = order_of_convergence(false_pos_errors_g)

nr_order_h = order_of_convergence(nr_errors_h)
secant_order_h = order_of_convergence(secant_errors_h)
bisection_order_h = order_of_convergence(bisection_errors_h)
false_pos_order_h = order_of_convergence(false_pos_errors_h)

# Print results
print("Newton-Raphson order of convergence for f(x):", nr_order_f)
print("Secant order of convergence for f(x):", secant_order_f)
print("Bisection order of convergence for f(x):", bisection_order_f)
print("False Position order of convergence for f(x):", false_pos_order_f)

print("Newton-Raphson order of convergence for g(x):", nr_order_g)
print("Secant order of convergence for g(x):", secant_order_g)
print("Bisection order of convergence for g(x):", bisection_order_g)
print("False Position order of convergence for g(x):", false_pos_order_g)

print("Newton-Raphson order of convergence for h(x):", nr_order_h)
print("Secant order of convergence for h(x):", secant_order_h)
print("Bisection order of convergence for h(x):", bisection_order_h)
print("False Position order of convergence for h(x):", false_pos_order_h)


Newton-Raphson order of convergence for f(x): 1.6243856064940572
Secant order of convergence for f(x): 1.109648581256464
Bisection order of convergence for f(x): 0.9428571428571428
False Position order of convergence for f(x): 0.9799999858452263
Newton-Raphson order of convergence for g(x): -0.38824893823528284
Secant order of convergence for g(x): 6.441425745327228
Bisection order of convergence for g(x): 0.9285714285714286
False Position order of convergence for g(x): 0.8056632322230998
Newton-Raphson order of convergence for h(x): 0.9828091366051239
Secant order of convergence for h(x): 1.0211415467570253
Bisection order of convergence for h(x): 0.9310344827586207
False Position order of convergence for h(x): 0.8949347223410226
